In [1]:
%load_ext autoreload
%autoreload 2
from EventDriven.riskmanager.utils import populate_cache_with_chain, get_persistent_cache
from EventDriven.riskmanager.picker import OrderSchema, build_strategy, filter_contracts
from trade.helpers.helper import retrieve_timeseries
def get_spot(tick, date, spot_type='close'):
    """
    Retrieves the spot price for a given ticker on a specific date.
    """
    return retrieve_timeseries(tick, date, date, spot_type=spot_type)['close'][0]



Console Logging & File Logging Can be configured using STREAM_LOG_LEVEL and FILE_LOG_LEVEL in environment variables.
Propagate to root logger can be set using PROPAGATE_TO_ROOT_LOGGER in environment variables.
Example:
STREAM_LOG_LEVEL = 'DEBUG'
FILE_LOG_LEVEL = 'INFO'
PROPAGATE_TO_ROOT_LOGGER = 'False'

2025-10-27 22:09:23 trade.helpers.Logging INFO: Logging Root Directory: /Users/chiemelienwanisobi/cloned_repos/QuantTools/logs
Using Proxy URL: http://54.205.248.219:5500/thetadata


Scheduled Data Requests will be saved to: /Users/chiemelienwanisobi/cloned_repos/QuantTools/module_test/raw_code/DataManagers/scheduler/requests.jsonl
2025-10-27 22:09:46 DataManager.py CRITICAL: Using ProcessSaveManager for saving data.


In [ ]:
from abc import ABC, abstractmethod
from trade.helpers.Logging import setup_logger
from pydantic.dataclasses import dataclass as pydantic_dataclass
import pandas as pd
from pandas.tseries.offsets import BDay
from typing import ClassVar
from weakref import WeakSet

logger = setup_logger(__name__, stream_log_level="DEBUG")

CONFIG_DEFINITIONS = {
    # 'Config class: { config_name}': 'Description of what this config class is for and how to use it.'
    'ChainConfigs': {
        'max_chain_size': 'Maximum number of option contracts to retrieve in a single chain.',
        'enable_caching': 'Flag to enable or disable caching of option chains to improve performance.',
    },
}

@pydantic_dataclass
class BaseConfigs(ABC):
    """Base configuration class for all modules."""


    _registry: ClassVar[WeakSet[type]] = WeakSet()

    def __init_subclass__(cls, **kwargs):
        super().__init_subclass__(**kwargs)
        BaseConfigs._registry.add(cls)

    def display_configs(self):
        """Display the current configuration settings."""
        msg = f"""
        Current Configuration Settings for {self.__class__.__name__}:
        {self.__dict__}
        """
        return msg

    def describe_configs(self):
        """Describe the configuration settings with explanations."""

        class_descriptions = CONFIG_DEFINITIONS.get(self.__class__.__name__)
        if not class_descriptions:
            logger.warning(f"No configuration descriptions found for {self.__class__.__name__}.")
            return
        header = f"""
        Configuration Descriptions for {self.__class__.__name__}:
        """
        
        msg = ''
        for key, value in self.__dict__.items():
            desc = class_descriptions.get(key)
            if desc:
                msg += f"{' ' * 8}{key}: {value}  # {desc}\n"
            else:
                logger.warning(f"No description found for config '{key}' in {self.__class__.__name__}.")
        return header + msg

    def display_and_describe_configs(self):
        """Display and describe the configuration settings."""
        msg1=self.display_configs()
        msg2=self.describe_configs()
        print(msg1)
        print(msg2)

    def get_all_configs(cls):
        """Get all configuration classes and their instances."""
        configs = {}
        for config_cls in cls._registry:
            configs[config_cls.__name__] = config_cls()
        return configs


Console Logging & File Logging Can be configured using STREAM_LOG_LEVEL and FILE_LOG_LEVEL in environment variables.
Propagate to root logger can be set using PROPAGATE_TO_ROOT_LOGGER in environment variables.
Example:
STREAM_LOG_LEVEL = 'DEBUG'
FILE_LOG_LEVEL = 'INFO'
PROPAGATE_TO_ROOT_LOGGER = 'False'

2025-10-27 23:01:31 trade.helpers.Logging INFO: Logging Root Directory: /Users/chiemelienwanisobi/cloned_repos/QuantTools/logs


In [3]:
@pydantic_dataclass
class ChainConfigs(BaseConfigs):
    """Configuration class for Chain module."""

    # Example configuration parameters
    max_chain_size: int = 1000
    enable_caching: bool = True
    cache_expiry_minutes: int = 60

chain_conf = ChainConfigs()
chain_conf.display_and_describe_configs()

Registered config subclass: ChainConfigs
2025-10-27 23:02:18 __main__ WARNING: No description found for config 'cache_expiry_minutes' in ChainConfigs.

        Current Configuration Settings for ChainConfigs:
        {'max_chain_size': 1000, 'enable_caching': True, 'cache_expiry_minutes': 60}
        

        Configuration Descriptions for ChainConfigs:
                max_chain_size: 1000  # Maximum number of option contracts to retrieve in a single chain.
        enable_caching: True  # Flag to enable or disable caching of option chains to improve performance.



In [4]:
ChainConfigs._registry

{<weakref at 0x112cb0310; to 'ABCMeta' at 0x7fd2dc764e80 (ChainConfigs)>, <weakref at 0x112cb1c60; to 'ABCMeta' at 0x7fd2d7dee810 (ChainConfigs)>}

In [4]:
schema = OrderSchema(dict(
        target_dte= 270,
    strategy= 'vertical',
    structure_direction= 'long',
    spread_ticks= 1,
    dte_tolerance= 60,
    min_moneyness= 0.65,
    max_moneyness= 1,
    min_total_price= 0.95,
    option_type = 'c',
    max_total_price= 4,
    tick = 'AAPL',
))

In [5]:
get_persistent_cache().clear()
chain = populate_cache_with_chain(
    "AAPL", "2025-10-20"
)
chain

http://127.0.0.1:25510/v2/bulk_at_time/option/quote?root=AAPL&exp=0&start_date=20251020&end_date=20251020&ivl=57600000&use_csv=true&right=C
http://127.0.0.1:25510/v2/bulk_at_time/option/quote?root=AAPL&exp=0&start_date=20251020&end_date=20251020&ivl=57600000&use_csv=true&right=C
2025-10-27 22:09:48 QuantTools.EventDriven.riskmanager ERROR: Retrieved chain for AAPL on 2025-10-20
http://127.0.0.1:25510/v2/bulk_hist/option/open_interest?root=AAPL&exp=0&start_date=20251017&end_date=20251017&use_csv=true
http://127.0.0.1:25510/v2/bulk_hist/option/open_interest?root=AAPL&exp=0&start_date=20251017&end_date=20251017&use_csv=true
http://54.205.248.219:5500/thetadata


,datetime,root,expiration,strike,right,bid_size,closebid,ask_size,closeask,date,midpoint,weighted_midpoint,open_interest,opttick,chain_id,dte,pct_spread
0,2025-10-20,AAPL,2025-10-24,215.0,C,133,46.65,133,48.05,20251020,47.350,47.350000,232.0,AAPL20251121C325,AAPL20251121C325_2025-10-20,4,0.029567
1,2025-10-20,AAPL,2026-08-21,210.0,P,49,6.75,57,6.90,20251020,6.825,6.830660,734.0,AAPL20260618P250,AAPL20260618P250_2025-10-20,305,0.021978
2,2025-10-20,AAPL,2025-10-24,215.0,P,26,0.01,28,0.02,20251020,0.015,0.015185,818.0,AAPL20260220C165,AAPL20260220C165_2025-10-20,4,0.666667
3,2025-10-20,AAPL,2026-08-21,210.0,C,9,65.00,22,65.65,20251020,65.325,65.461290,59.0,AAPL20260918C130,AAPL20260918C130_2025-10-20,305,0.009950
4,2025-10-20,AAPL,2025-11-07,215.0,P,118,0.18,87,0.20,20251020,0.190,0.188488,575.0,AAPL20251114P325,AAPL20251114P325_2025-10-20,18,0.105263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2255,2025-10-20,AAPL,2028-01-21,200.0,P,11,13.45,18,13.70,20251020,13.575,13.605172,346.0,AAPL20280121P200,AAPL20280121P200_2025-10-20,823,0.018416
2256,2025-10-20,AAPL,2026-05-15,210.0,P,43,4.45,67,4.55,20251020,4.500,4.510909,1682.0,AAPL20260515P210,AAPL20260515P210_2025-10-20,207,0.022222
2257,2025-10-20,AAPL,2026-05-15,210.0,C,9,60.95,8,61.60,20251020,61.275,61.255882,1192.0,AAPL20260515C210,AAPL20260515C210_2025-10-20,207,0.010608
2258,2025-10-20,AAPL,2026-06-18,210.0,C,7,62.40,7,62.95,20251020,62.675,62.675000,5799.0,AAPL20260618C210,AAPL20260618C210_2025-10-20,241,0.008775


In [6]:
from dbase.DataAPI.ThetaData import retrieve_chain_bulk, retrieve_bulk_open_interest
tick = schema['tick']
date = '2025-09-23'
print_url = True
chain = retrieve_chain_bulk(
    tick,
    '',
    date,
    date,
    '16:00',
    'C',
    print_url = print_url
)



    # ## Clip Chain
    # chain_clipped = chain.reset_index()[['datetime', 'Root', 'Strike', 'Right', 'Expiration', 'Midpoint']]
    # if PATCH_TICKERS:
    #     chain_clipped['Root'] = chain_clipped['Root'].apply(swap_ticker)

    # ## Create ID
    # id_params = chain_clipped[['Root', 'Right', 'Expiration', 'Strike']].T.to_numpy()
    # ids = runThreads(
    #     generate_option_tick_new, 
    #     id_params)
    # chain_clipped['opttick'] = ids
    # filter_opt = get_avoid_opticks(tick)
    # chain_clipped = chain_clipped[~chain_clipped['opttick'].isin(filter_opt)] ## Optticks to avoid
    # chain_clipped['chain_id'] = chain_clipped['opttick'] + '_' + chain_clipped['datetime'].astype(str)
    # chain_clipped['dte'] = (pd.to_datetime(chain_clipped['Expiration']) - pd.to_datetime(chain_clipped['datetime'])).dt.days

    # ## Save to cache
    # def save_to_cache(id, date, spot):
    #     date = pd.to_datetime(date).strftime('%Y-%m-%d')
    #     save_id = f"{id}_{date}"
    #     if save_id not in get_cache('spot'):
    #         spot_cache[save_id] = spot
    # save_params = chain_clipped[['opttick', 'datetime', 'Midpoint']].T.to_numpy()
    # runThreads(
    #     save_to_cache, 
    #     save_params)
    
    # if chain_spot:
    #     chain_clipped['spot']=chain_spot
    #     chain_clipped['moneyness']=0
    #     chain_clipped.loc[chain_clipped['Right'] == 'C', 'moneyness'] = chain_clipped.loc[chain_clipped['Right'] == 'C', 'Strike'] / chain_clipped.loc[chain_clipped['Right'] == 'C', 'spot']
    #     chain_clipped.loc[chain_clipped['Right'] == 'P', 'moneyness'] = chain_clipped.loc[chain_clipped['Right'] == 'P', 'spot'] / chain_clipped.loc[chain_clipped['Right'] == 'P', 'Strike']
    #     chain_clipped=chain_clipped[chain_clipped['moneyness'].between(0.01, 3)] ## Filter out extreme moneyness to reduce size
    # chain_clipped.columns = chain_clipped.columns.str.lower()'
chain

http://127.0.0.1:25510/v2/bulk_at_time/option/quote?root=AAPL&exp=0&start_date=20250923&end_date=20250923&ivl=57600000&use_csv=true&right=C
http://127.0.0.1:25510/v2/bulk_at_time/option/quote?root=AAPL&exp=0&start_date=20250923&end_date=20250923&ivl=57600000&use_csv=true&right=C


,Root,Expiration,Strike,Right,Bid_size,CloseBid,Ask_size,CloseAsk,Date,Midpoint,Weighted_midpoint
datetime,,,,,,,,,,,
2025-09-23,AAPL,2025-10-03,215.0,P,123,0.04,144,0.06,20250923,0.050,0.050787
2025-09-23,AAPL,2025-09-26,215.0,P,0,0.00,10,0.01,20250923,0.005,0.010000
2025-09-23,AAPL,2025-09-26,215.0,C,214,38.55,52,39.90,20250923,39.225,38.813910
2025-09-23,AAPL,2025-10-03,215.0,C,157,38.80,60,40.20,20250923,39.500,39.187097
2025-09-23,AAPL,2025-10-10,215.0,P,63,0.15,63,0.17,20250923,0.160,0.160000
...,...,...,...,...,...,...,...,...,...,...,...
2025-09-23,AAPL,2028-01-21,200.0,C,8,82.15,44,83.60,20250923,82.875,83.376923
2025-09-23,AAPL,2026-05-15,210.0,C,57,54.70,82,55.55,20250923,55.125,55.201439
2025-09-23,AAPL,2026-05-15,210.0,P,52,5.40,21,5.55,20250923,5.475,5.443151


In [7]:
## Get open interest
##  Because open interest is only updated EOD, we retrieve previous day's open interest
prev = (pd.to_datetime(date) - BDay(1)).strftime('%Y-%m-%d')
oi = retrieve_bulk_open_interest(
    symbol = tick,
    exp = 0,
    start_date = prev,
    end_date = prev,
    print_url = True
)
oi

http://127.0.0.1:25510/v2/bulk_hist/option/open_interest?root=AAPL&exp=0&start_date=20250922&end_date=20250922&use_csv=true
http://127.0.0.1:25510/v2/bulk_hist/option/open_interest?root=AAPL&exp=0&start_date=20250922&end_date=20250922&use_csv=true
http://54.205.248.219:5500/thetadata


,Root,Expiration,Strike,Right,Open_interest,Date,time,Datetime
0,AAPL,2025-10-03,215.0,P,1246,20250922,06:30:15,2025-09-22
1,AAPL,2025-10-03,215.0,C,631,20250922,06:30:19,2025-09-22
2,AAPL,2026-08-21,210.0,C,43,20250922,06:30:15,2025-09-22
3,AAPL,2026-08-21,210.0,P,559,20250922,06:30:19,2025-09-22
4,AAPL,2025-10-17,215.0,C,13676,20250922,06:30:04,2025-09-22
...,...,...,...,...,...,...,...,...
2235,AAPL,2026-01-16,210.0,P,36400,20250922,06:30:08,2025-09-22
2236,AAPL,2025-09-26,212.5,C,56,20250922,06:30:02,2025-09-22
2237,AAPL,2025-09-26,212.5,P,406,20250922,06:30:07,2025-09-22
2238,AAPL,2027-12-17,200.0,C,4911,20250922,06:30:08,2025-09-22


In [8]:
chain =chain.merge(oi[['Root', 'Expiration', 'Strike', 'Right', 'Open_interest']], on=['Root', 'Expiration', 'Strike', 'Right'], how='left')


In [9]:
oi_thresholds = [25, 50, 100, 200, 500, 1000]
spread_threshold = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3]

In [23]:
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'NVDA', 'JNJ', 'V', 'WMT', 'NFLX', 'SBUX', 'BA', 'AMD', 'COST']
main_chain = {}
documenting = {}

for tick in tickers:
    documenting[tick] = {}
    logger.info(f"Processing ticker: {tick}")
    date = '2025-08-20'
    print_url = True

    chain = retrieve_chain_bulk(
        tick,
        '',
        date,
        date,
        '16:00',
        'C',
        print_url = print_url
    )

    prev = (pd.to_datetime(date) - BDay(1)).strftime('%Y-%m-%d')
    oi = retrieve_bulk_open_interest(
        symbol = tick,
        exp = 0,
        start_date = prev,
        end_date = prev,
        print_url = False
    )
    oi

    chain =chain.merge(oi[['Root', 'Expiration', 'Strike', 'Right', 'Open_interest']], on=['Root', 'Expiration', 'Strike', 'Right'], how='left')
    chain['spread'] = chain['CloseAsk'] - chain['CloseBid']
    chain['pct_spread'] = chain['spread'] / ((chain['CloseAsk'] + chain['CloseBid']) / 2)
    chain['DTE'] = (pd.to_datetime(chain['Expiration']) - pd.to_datetime(date)).dt.days
    print(f"Max DTE: {chain['DTE'].max()} days")

    ## Sizes after filtering for different pct_spread thresholds
    print(f"Sizes after filtering for different pct_spread thresholds. Total original size: {len(chain)}")
    for threshold in spread_threshold:
        main_chain[tick] = chain.copy() 
        documenting[tick]['chain_size'] = len(chain)
        filtered_chain = chain[chain['pct_spread'] <= threshold]
        print(f"Threshold: {threshold:.2%}, Size: {len(filtered_chain)}")
        
        # print("Size by Expiration Change:")
        original_sizes = chain.groupby('DTE').size()
        filtered_sizes = filtered_chain.groupby('DTE').size()
        size_comparison = pd.DataFrame({
            'Original Size': original_sizes,
            'Filtered Size': filtered_sizes,
            'Reduction': original_sizes - filtered_sizes,
            'pct Reduction': (original_sizes - filtered_sizes) / original_sizes * 100
        }).fillna(0).astype(int)
        documenting[tick][f'spread_{threshold}_oi_base'] = filtered_chain.shape[0]

        for oi_threshold in oi_thresholds:
            print(f"  Sizes after applying Open Interest threshold of {oi_threshold}: {filtered_chain[filtered_chain['Open_interest'] > oi_threshold].shape[0]}")
            documenting[tick][f'spread_{threshold}_oi_{oi_threshold}'] = filtered_chain[filtered_chain['Open_interest'] > oi_threshold].shape[0]
        # print(size_comparison)


2025-10-27 22:25:47 __main__ INFO: Processing ticker: AAPL
http://127.0.0.1:25510/v2/bulk_at_time/option/quote?root=AAPL&exp=0&start_date=20250820&end_date=20250820&ivl=57600000&use_csv=true&right=C
http://127.0.0.1:25510/v2/bulk_at_time/option/quote?root=AAPL&exp=0&start_date=20250820&end_date=20250820&ivl=57600000&use_csv=true&right=C
Max DTE: 849 days
Sizes after filtering for different pct_spread thresholds. Total original size: 2394
Threshold: 5.00%, Size: 1718
  Sizes after applying Open Interest threshold of 25: 1118
  Sizes after applying Open Interest threshold of 50: 1062
  Sizes after applying Open Interest threshold of 100: 987
  Sizes after applying Open Interest threshold of 200: 865
  Sizes after applying Open Interest threshold of 500: 692
  Sizes after applying Open Interest threshold of 1000: 564
Threshold: 10.00%, Size: 1890
  Sizes after applying Open Interest threshold of 25: 1260
  Sizes after applying Open Interest threshold of 50: 1198
  Sizes after applying Ope

In [25]:
df = pd.DataFrame(documenting)
df_no_chain = df.drop(index=['chain_size'])
df_chain = df.loc['chain_size']
(df_no_chain/df_chain.T).median(axis=1)

spread_0.05_oi_base    0.612684
spread_0.05_oi_25      0.341986
spread_0.05_oi_50      0.301065
spread_0.05_oi_100     0.257661
spread_0.05_oi_200     0.208259
spread_0.05_oi_500     0.142005
spread_0.05_oi_1000    0.093250
spread_0.1_oi_base     0.720243
spread_0.1_oi_25       0.408861
spread_0.1_oi_50       0.368198
spread_0.1_oi_100      0.321277
spread_0.1_oi_200      0.268557
spread_0.1_oi_500      0.185056
spread_0.1_oi_1000     0.121374
spread_0.15_oi_base    0.752023
spread_0.15_oi_25      0.450773
spread_0.15_oi_50      0.406846
spread_0.15_oi_100     0.356866
spread_0.15_oi_200     0.295974
spread_0.15_oi_500     0.201309
spread_0.15_oi_1000    0.133942
spread_0.2_oi_base     0.770896
spread_0.2_oi_25       0.475521
spread_0.2_oi_50       0.431435
spread_0.2_oi_100      0.378336
spread_0.2_oi_200      0.310677
spread_0.2_oi_500      0.210211
spread_0.2_oi_1000     0.140855
spread_0.25_oi_base    0.785181
spread_0.25_oi_25      0.497482
spread_0.25_oi_50      0.451427
spread_0

In [31]:
order_cache = {}
orders = {}
schema['max_total_price']=5
for chain in main_chain.values():
    logger.info(f"Building order for ticker: {chain['Root'].iloc[0]}")
    schema['tick']=chain['Root'].iloc[0]
    copy_chain = chain.copy()
    copy_chain.columns=copy_chain.columns.str.lower()
    # copy_chain = copy_chain[copy_chain['pct_spread'] <= 0.10]
    # copy_chain = copy_chain[copy_chain['open_interest'] > 25]
    print(f"Cahin size after filtering: {len(copy_chain)}")
    order = build_strategy(
        copy_chain,
        schema=schema,
        spot = get_spot(schema['tick'], date, 'chain_close'),
        cache=order_cache
    )
    if not order:
        logger.warning(f"No order generated for ticker: {schema['tick']}")
        continue
    order[0].pop('legs')
    order[0].pop('long')
    order[0].pop('short')
    orders[schema['tick']] = order[0]

2025-10-27 22:38:04 __main__ INFO: Building order for ticker: AAPL
Cahin size after filtering: 2394
2025-10-27 22:38:04 __main__ INFO: Building order for ticker: MSFT
Cahin size after filtering: 3424
2025-10-27 22:38:04 __main__ INFO: Building order for ticker: GOOGL
Cahin size after filtering: 2122
2025-10-27 22:38:04 __main__ INFO: Building order for ticker: AMZN
Cahin size after filtering: 2136
2025-10-27 22:38:04 __main__ INFO: Building order for ticker: TSLA
Cahin size after filtering: 4256
2025-10-27 22:38:04 __main__ INFO: Building order for ticker: NVDA
Cahin size after filtering: 4016
2025-10-27 22:38:04 __main__ INFO: Building order for ticker: JNJ
Cahin size after filtering: 1132
2025-10-27 22:38:04 __main__ INFO: Building order for ticker: V
Cahin size after filtering: 2584
2025-10-27 22:38:04 __main__ INFO: Building order for ticker: WMT
Cahin size after filtering: 1192
2025-10-27 22:38:05 __main__ INFO: Building order for ticker: NFLX
Cahin size after filtering: 7142
2025

In [32]:
stuff = pd.DataFrame(orders).T
stuff['long_min_short_bid'] = stuff['long_bid'] - stuff['short_bid']
stuff['long_min_short_ask'] = stuff['long_ask'] - stuff['short_ask']
stuff[['long_bid', 'long_ask', 'short_bid', 'short_ask', 'long_min_short_bid', 'long_min_short_ask', 'spread_bid', 'spread_ask', 'spread_mid', 'spread_pct_ratio']]

,long_bid,long_ask,short_bid,short_ask,long_min_short_bid,long_min_short_ask,spread_bid,spread_ask,spread_mid,spread_pct_ratio
AAPL,49.15,49.35,45.05,45.2,4.1,4.15,3.95,4.3,4.125,0.021212
MSFT,74.5,75.05,70.8,71.3,3.7,3.75,3.2,4.25,3.725,0.07047
GOOGL,47.0,47.25,43.0,43.2,4.0,4.05,3.8,4.25,4.025,0.02795
AMZN,27.65,27.85,24.75,24.9,2.9,2.95,2.75,3.1,2.925,0.029915
TSLA,77.75,78.1,72.9,73.35,4.85,4.75,4.4,5.2,4.8,0.041667
NVDA,26.75,27.0,24.25,24.3,2.5,2.7,2.45,2.75,2.6,0.028846
JNJ,26.75,27.15,22.55,22.95,4.2,4.2,3.8,4.6,4.2,0.047619
V,50.9,52.3,47.15,48.75,3.75,3.55,2.15,5.15,3.65,0.205479
WMT,23.1,23.4,20.95,21.25,2.15,2.15,1.85,2.45,2.15,0.069767
NFLX,153.45,155.3,150.6,152.5,2.85,2.8,0.95,4.7,2.825,0.331858


In [19]:
stuff = pd.DataFrame(orders).T
stuff['long_min_short_bid'] = stuff['long_bid'] - stuff['short_bid']
stuff['long_min_short_ask'] = stuff['long_ask'] - stuff['short_ask']
stuff[['long_bid', 'long_ask', 'short_bid', 'short_ask', 'long_min_short_bid', 'long_min_short_ask', 'spread_bid', 'spread_ask', 'spread_mid', 'spread_pct_ratio']]

,long_bid,long_ask,short_bid,short_ask,long_min_short_bid,long_min_short_ask,spread_bid,spread_ask,spread_mid,spread_pct_ratio
AAPL,34.6,34.8,31.4,31.65,3.2,3.15,2.95,3.4,3.175,0.035433
MSFT,80.55,81.15,77.15,77.7,3.4,3.45,2.85,4.0,3.425,0.083942
GOOGL,59.85,60.15,56.15,56.6,3.7,3.55,3.25,4.0,3.625,0.051724
AMZN,61.75,62.15,57.85,58.1,3.9,4.05,3.65,4.3,3.975,0.040881
TSLA,94.95,96.35,90.65,91.5,4.3,4.85,3.45,5.7,4.575,0.122951
NVDA,35.8,36.0,32.85,33.05,2.95,2.95,2.75,3.15,2.95,0.033898
JNJ,26.5,26.85,22.65,22.95,3.85,3.9,3.55,4.2,3.875,0.041935
V,42.3,42.9,39.0,39.55,3.3,3.35,2.75,3.9,3.325,0.086466
WMT,17.2,17.35,13.6,14.1,3.6,3.25,3.1,3.75,3.425,0.047445
SBUX,23.3,23.55,19.05,19.7,4.25,3.85,3.6,4.5,4.05,0.055556


In [135]:
chain_lower_cols = main_chain['NFLX'].copy()
chain_lower_cols.columns=chain_lower_cols.columns.str.lower()
filtered = filter_contracts(
    df=chain_lower_cols,
    schema=schema,
    spot=get_spot('NFLX', date, 'chain_close'),

)
filtered[(filtered['pct_spread'] <= 0.075) & (filtered['open_interest'] > 25) ]

,root,expiration,strike,right,bid_size,closebid,ask_size,closeask,date,midpoint,weighted_midpoint,open_interest,spread,pct_spread,dte
7,NFLX,2026-06-18,740.0,P,3,11.35,7,12.00,20251023,11.675,11.805000,80.0,0.65,0.055675,238
11,NFLX,2026-09-18,740.0,P,8,18.30,10,19.30,20251023,18.800,18.855556,31.0,1.00,0.053191,330
12,NFLX,2026-06-18,750.0,C,3,395.20,5,401.05,20251023,398.125,398.856250,36.0,5.85,0.014694,238
13,NFLX,2026-06-18,750.0,P,6,12.20,10,12.95,20251023,12.575,12.668750,345.0,0.75,0.059642,238
18,NFLX,2026-06-18,760.0,P,6,13.10,12,13.90,20251023,13.500,13.633333,194.0,0.80,0.059259,238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,NFLX,2026-06-18,1100.0,P,7,98.95,7,101.75,20251023,100.350,100.350000,606.0,2.80,0.027902,238
226,NFLX,2026-09-18,1100.0,P,10,117.15,5,119.95,20251023,118.550,118.083333,188.0,2.80,0.023619,330
228,NFLX,2026-06-18,1110.0,P,7,103.60,7,106.45,20251023,105.025,105.025000,106.0,2.85,0.027136,238
229,NFLX,2026-06-18,1110.0,C,6,136.85,15,140.85,20251023,138.850,139.707143,26.0,4.00,0.028808,238


In [94]:
chain_lower_cols#.dte.min()

,root,expiration,strike,right,bid_size,closebid,ask_size,closeask,date,midpoint,weighted_midpoint,open_interest,spread,pct_spread,dte
0,AAPL,2026-08-21,210.0,P,30,7.00,24,7.15,20251023,7.075,7.066667,734.0,0.15,0.021201,20686
1,AAPL,2026-08-21,210.0,C,5,62.55,6,63.05,20251023,62.800,62.822727,63.0,0.50,0.007962,20685
2,AAPL,2025-10-24,215.0,C,173,43.95,160,45.20,20251023,44.575,44.550601,250.0,1.25,0.028043,20384
3,AAPL,2025-10-31,215.0,C,162,44.05,127,45.60,20251023,44.825,44.731142,179.0,1.55,0.034579,20391
4,AAPL,2025-10-24,215.0,P,0,0.00,6,0.01,20251023,0.005,0.010000,1639.0,0.01,2.000000,20384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2341,AAPL,2028-01-21,200.0,C,7,86.80,7,87.75,20251023,87.275,87.275000,582.0,0.95,0.010885,21203
2342,AAPL,2026-05-15,210.0,P,32,4.65,23,4.75,20251023,4.700,4.691818,1849.0,0.10,0.021277,20587
2343,AAPL,2026-05-15,210.0,C,15,58.35,8,58.95,20251023,58.650,58.558696,1197.0,0.60,0.010230,20587
2344,AAPL,2026-06-18,210.0,C,2,59.90,4,60.30,20251023,60.100,60.166667,5761.0,0.40,0.006656,20621


In [83]:
orders

{'AAPL': [],
 'MSFT': [],
 'GOOGL': [],
 'AMZN': [],
 'TSLA': [],
 'NVDA': [],
 'JNJ': [],
 'V': [],
 'WMT': []}